# FrozenLake Experiments (from `main.py`)

- The assignment does not explicitly specify whether the environment transitions are deterministic or stochastic. 

- However, since the Frozen Lake problem can also be interpreted as a slippery environment, experiments were conducted under both settings.
- Deterministic Transition Environment: "frozen_lake_env.py"
- Stochastic Transition (Slippery Environment): "environment_probablistic.py"

- In the probabilistic Frozen Lake environment, slipping is modeled through stochastic action execution. The intended action is executed with probability 1-p, while with probability p, one of the two perpendicular actions is executed with equal probability. Rewards remain deterministic.

For example: If agent chooses RIGHT:
- executes RIGHT with prob 0.8 ---- executes UP with prob 0.1 --- executes DOWN with prob 0.1




### Task 1: Training using Monte Carlo Reinforcement Technique

- Experiment 1: Deterministic Environment - 4×4 grid - Epsilon is fixed - Tie Breaking is biased towards lower indexed action (Executed from monte_carlo.py) - Training upto 30k episodes ---- Outcome: Success rate converges to 1 approximately after 9000 epsisodes

- Experiment 2: Extended version of Experiment 1 for 10×10 grid - Training upto 80k episodes ---- Outcome: Success rate remains 0 throughout the training, Robot osicallates between 2 cells inorder to avoid falling into the hole which produces the negative reward

- Experiment 3: Probablistic Environment - 4×4 grid - Same assumption as Experiment 1 ---- Outcome: success rate Converges approximately to 0.8 (which means within the tested 200 episodes: approximately 160 episodes reaches the goal successfully)

- Experiment 4: Extended version of Experiment 3 for 4×4 grid with more training upto 80k epsisodes ---- Outcome: Observations are similar to Experiment 3

- Experiment 5: Extended version of Experiment 4 for 10×10 grid ---- Outcome: Observations are similar to Experiment 2 - Success rate remains 0 throughout the training

-------------------------------------------------------------------------------------------------------

- Experiment 6: Deterministic Environment - 4×4 grid - exponentially decaying ε-greedy strategy was adopted - Random tie-breaking was used for greedy action selection to avoid systematic bias toward lower-indexed actions (Left side) - (Executed through monte_carlo_update.py) ---- Outcome: Success rate converges to 1 from very early training (Within 3000 episodes of Training)

- Experiment 7: Extended version of Experiment 6 on a 10×10 grid (with deterministic transitions, random tie-breaking, and a decaying ε-greedy strategy. ) ---- Outcome: Despite these algorithmic improvements, the method failed to converge, as the greedy policy success rate remained zero even after 80,000 training episodes

- Experiment 8: Probablistic Environment - 4×4 grid - Same assumption as Experiment 6 ---- Outcome: Observations are similar to Experiment 3 and 4 (success rate Converges approximately to 0.8)

- Experiment 9: Extended version of Experiment for 10×10 grid ---- Outcome: Observations are similar to Experiment 2 - Success rate remains 0 throughout the training  

### Task 2:  Training using SARSA with an ϵ-greedy behavior policy

SARSA: Unlike Monte Carlo methods, SARSA does not require waiting for an episode to terminate to update value estimates. Instead, the action-value function is updated at every time step using the immediate reward and the subsequent state–action pair (SARSA performs on-policy updates at every time step using the immediate reward and the next state–action pair selected by the behavior policy).

- Experiment 10: Deterministic Environment - 4×4 grid - Random tie-breaking - Fixed Epsilon ---- Outcome: Success rate converges to 1 from very early training (Within 3000 episodes of Training) 

- Experiment 11: Deterministic Environment - 10×10 grid - Random tie-breaking - Decaying Epsilon (Initial Epsilon 0.6) ---- Outcome: Able to see the success rate 1 for the first time in 10×10 grid at the very end of the training

- Experiment 12: Deterministic Environment - 10×10 grid - Random tie-breaking - Decaying Epsilon - But the inital epsilon value is lower than Experiment 11 (0.4) --- Outcome: SARSA sometimes produces a fully successful greedy policy - Then later loses it - Then recovers and stabilizes 

Reason: SARSA is based upon on-policy, Q value updated based on the action it took, So it gradually build a smooth value gradient toward the goal state. Make learning progress even when successful goal-reaching episodes are rare. 

- Experiment 13: Probablistic Transition (Slippery Environment) - 4×4 grid - Random tie-breaking - Fixed Epsilon ---- Outcome: Observations are almost similar to Monte carlo results, success converge between 0.75 – 0.85 which is bit higher than Monte-Carlo.

- Experiment 14: Probablistic Transition (Slippery Environment) - 10×10 grid - Random tie-breaking - Decaying Epsilon ---- Outcome: Despite prolonged training and ε decay, SARSA exhibits fluctuating greedy success rates; 

- Experiment 15: Probablistic Transition (Slippery Environment) - 10×10 grid - Random tie-breaking - Decaying Epsilon - But with more Training episodes ---- Outcome: Same as Experiment 14, couldn't able to observe much difference

### Reason why outcome differs from Monte Carlo (Reason for oscillation of success rate): 
- Since SARSA is an on-policy method, it learns Q-values under continued ε-greedy exploration and stochastic transitions, leading to frequent re-weighting of safe versus risky paths.


### Task 3:  𝑄-learning with an ϵ-greedy behavior policy.

Unlike Monte Carlo methods, Q-learning updates action values at every time step without waiting for episode termination, using immediate rewards and bootstrapped estimates. Unlike SARSA, it is an off-policy method: the update uses the maximum Q-value of the next state, independent of the action actually taken by the ε-greedy behavior policy. As a result, Q-learning learns the value of the greedy target policy, while exploration only affects experience generation.

- Experiment 16: Deterministic Environment - 4×4 grid - Random tie-breaking - Fixed Epsilon ---- Outcome: output looks much similar to Monte Carlo and SARSA, output looks much similar to Monte Carlo and SARSA

- Experiment 17: Deterministic Environment - 10×10 grid  - Random tie-breaking - Decaying Epsilon ---- Outcome: Converge to 1 within the few Training episodes (less than 3000)

- Experiment 18: Probablistic Transition (Slippery Environment) - 4×4 grid - Random tie-breaking - Fixed Epsilon ---- Outcome: There is no single path from Start to Goal, loops (L <-> Boundry patterns)

- Experiment 19: Probablistic Transition (Slippery Environment) - 10×10 grid - Random tie-breaking - Decaying Epsilon ---- Outcome: Does not converge even for a small value, remains 0 even after 16k Training episodes

In [1]:
%matplotlib tk


In [2]:
# --- General imports ---
import sys
import time
import matplotlib.pyplot as plt

In [3]:
# Deterministic environment
from frozen_lake_env import FrozenLakeEnv as FrozenLakeEnvDet, generate_random_solvable_holes

# Probabilistic environment
from environment_probablistic import FrozenLakeEnv as FrozenLakeEnvProb

# Renderer for Frozen Lake GUI
from frozen_lake_render import FrozenLakeMatplotlibRenderer

In [4]:
# --- Original Monte-Carlo imports (Task 1) ---
from monte_carlo import (
    mc_control_first_visit_no_exploring_starts as mc_control_base,
    greedy_action as greedy_action_base,
    print_greedy_policy_grid as print_policy_base,
    evaluate_greedy_policy as eval_greedy_base,
)

In [5]:
# --- Updated Monte-Carlo (random tie-break + epsilon decay) imports ---
from monte_carlo_update import (
    mc_control_first_visit_no_exploring_starts as mc_control_up,
    greedy_action as greedy_action_up,
    print_greedy_policy_grid as print_policy_up,
    evaluate_greedy_policy as eval_greedy_up,
)

In [23]:
# --- SARSA (Task 2) imports ---
from sarsa import (
    sarsa_control_epsilon_greedy as sarsa_control_up,
    greedy_action as greedy_action_sarsa,
    print_greedy_policy_grid as print_policy_sarsa,
    evaluate_greedy_policy as eval_greedy_sarsa,
)

In [44]:
# --- Q-Learning (Task 3) imports ---
from q_learning import (
    q_learning_control_epsilon_greedy as q_learning_control_up,
    greedy_action as greedy_action_q,
    print_greedy_policy_grid as print_policy_q,
    evaluate_greedy_policy as eval_greedy_q,
)


### Visualization of the single episode after Training

In [7]:
def run_episode_with_render(env, Q, renderer, greedy_action_fn, max_steps=200, pause=0.30):
    """
    Run ONE episode using the greedy policy from Q.
    Updates matplotlib renderer each step.
    """
    s = env.reset()

    print("Initial state_id:", s, "pos:", env.s)
    env.render()        # ASCII
    renderer.draw()     # GUI initial draw

    for t in range(max_steps):
        a = greedy_action_fn(Q[s])
        ns, r, done, info = env.step(a)

        # If probabilistic env, show executed action info too
        extra = ""
        if isinstance(info, dict) and "executed_action" in info:
            extra = f" | executed={env.ACTION_NAMES[info['executed_action']]} slipped={info.get('slipped', False)}"

        print(
            f"[t={t}] a={env.ACTION_NAMES[a]} ({a})  ns={ns}  r={r}  done={done}  pos={env.s}{extra}"
        )

        env.render()
        renderer.draw(action=a, reward=r, done=done)

        s = ns

        if pause is not None and pause > 0:
            time.sleep(pause)

        if done:
            print("Episode finished.")
            break

    print("Close plot window to exit.")
    plt.ioff()
    plt.show()

### Reusable pipeline for Monte-Carlo Experiments

- Environment setup (deterministic or probabilistic)
- Train a policy using Monte-Carlo control
- Evaluate the learned greedy policy
- Optionally visualize one greedy episode

In [ ]:
def run_mc_experiment(
    env_class,
    n,
    holes,
    env_seed,
    mc_control_fn,
    greedy_action_fn,
    print_policy_fn,
    eval_fn,
    mc_kwargs=None,
    # Common experiment settings:
    train_episodes=30000,
    gamma=0.99,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs=None,
):
    """
    Runs MC control on either deterministic or probabilistic environment,
    depending on env_class passed in.
    """
    env_kwargs = env_kwargs or {}
    mc_kwargs = mc_kwargs or {}

    print("\n" + "=" * 80)
    print(f"Running MC experiment: grid={n}x{n} | holes={len(holes)} | hole_ratio={len(holes)/(n*n):.2%}")
    print(f"Environment: {env_class.__name__} | extra_args={env_kwargs}")
    print(f"MC implementation: {mc_control_fn.__module__}.{mc_control_fn.__name__} | mc_kwargs={mc_kwargs}")
    print("=" * 80)

    env = env_class(n=n, holes=holes, seed=env_seed, **env_kwargs)

    # --- Train ---
    Q, pi = mc_control_fn(
        env,
        num_episodes=train_episodes,
        gamma=gamma,
        max_steps_per_episode=max_steps_train,
        seed=0,
        verbose_every=verbose_every,
        **mc_kwargs
    )

    # --- Print policy ---
    print("\nFinal greedy policy (grid):")
    print_policy_fn(env, Q)

    # --- Evaluate ---
    sr = eval_fn(env, Q, episodes=eval_episodes, max_steps=max_steps_eval, seed=999)
    print(f"\nFinal greedy success rate over {eval_episodes} episodes: {sr:.3f}")

    if render_one_episode:
        if env_class.__name__.lower().find("prob") != -1 or env_kwargs.get("slippery", False):
            env_label = "Slippery Environment"
        else:
            env_label = "Deterministic Transitions"

        renderer = FrozenLakeMatplotlibRenderer(
            env,
            bg_image_path=None,
            pause=render_pause,
            title=f"Greedy after MC {n}x{n} - {env_label}"
        )
        run_episode_with_render(
            env, Q, renderer, greedy_action_fn,
            max_steps=max_steps_eval, pause=render_pause
        )

### Reusable pipeline for SARSA Experiments

In [38]:
def run_sarsa_experiment(
    env_class,
    n,
    holes,
    env_seed,
    sarsa_control_fn,
    greedy_action_fn,
    print_policy_fn,
    eval_fn,
    sarsa_kwargs=None,
    # Common experiment settings:
    train_episodes=30000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs=None,
):
    """
    Runs SARSA control on either deterministic or probabilistic environment,
    depending on env_class passed in.
    """
    env_kwargs = env_kwargs or {}
    sarsa_kwargs = sarsa_kwargs or {}

    print("\n" + "=" * 80)
    print(f"Running SARSA experiment: grid={n}x{n} | holes={len(holes)} | hole_ratio={len(holes)/(n*n):.2%}")
    print(f"Environment: {env_class.__name__} | extra_args={env_kwargs}")
    print(f"SARSA implementation: {sarsa_control_fn.__module__}.{sarsa_control_fn.__name__} | sarsa_kwargs={sarsa_kwargs}")
    print("=" * 80)

    env = env_class(n=n, holes=holes, seed=env_seed, **env_kwargs)

    # --- Train ---
    Q, pi = sarsa_control_fn(
        env,
        num_episodes=train_episodes,
        gamma=gamma,
        alpha=alpha,
        max_steps_per_episode=max_steps_train,
        seed=0,
        verbose_every=verbose_every,
        **sarsa_kwargs
    )

    # --- Print policy ---
    print("\nFinal greedy policy (grid):")
    print_policy_fn(env, Q)

    # --- Evaluate ---
    sr = eval_fn(env, Q, episodes=eval_episodes, max_steps=max_steps_eval, seed=999)
    print(f"\nFinal greedy success rate over {eval_episodes} episodes: {sr:.3f}")

     # --- Render one episode (optional) ---
    if render_one_episode:
        if env_class.__name__.lower().find("prob") != -1 or env_kwargs.get("slippery", False):
            env_label = "Slippery Environment"
        else:
            env_label = "Deterministic Transitions"

        renderer = FrozenLakeMatplotlibRenderer(
            env,
            bg_image_path=None,
            pause=render_pause,
            title=f"Greedy after SARSA {n}x{n} - {env_label}"
        )
        run_episode_with_render(
            env, Q, renderer, greedy_action_fn,
            max_steps=max_steps_eval, pause=render_pause
        )

### Reusable pipeline for Q-learning Experiments

In [41]:
def run_q_learning_experiment(
    env_class,
    n,
    holes,
    env_seed,
    q_learning_control_fn,
    greedy_action_fn,
    print_policy_fn,
    eval_fn,
    q_learning_kwargs=None,
    # Common experiment settings:
    train_episodes=30000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs=None,
):
    """
    Runs Q-learning control on either deterministic or probabilistic environment,
    depending on env_class passed in.

    Behavior policy: epsilon-greedy (for action selection during training)
    Target for update: greedy max_a' Q(S',a')  (off-policy)
    """
    env_kwargs = env_kwargs or {}
    q_learning_kwargs = q_learning_kwargs or {}

    print("\n" + "=" * 80)
    print(f"Running Q-learning experiment: grid={n}x{n} | holes={len(holes)} | hole_ratio={len(holes)/(n*n):.2%}")
    print(f"Environment: {env_class.__name__} | extra_args={env_kwargs}")
    print(f"Q-learning implementation: {q_learning_control_fn.__module__}.{q_learning_control_fn.__name__} | q_learning_kwargs={q_learning_kwargs}")
    print("=" * 80)

    env = env_class(n=n, holes=holes, seed=env_seed, **env_kwargs)

    # --- Train ---
    Q, pi = q_learning_control_fn(
        env,
        num_episodes=train_episodes,
        gamma=gamma,
        alpha=alpha,
        max_steps_per_episode=max_steps_train,
        seed=0,
        verbose_every=verbose_every,
        **q_learning_kwargs
    )

    # --- Print policy ---
    print("\nFinal greedy policy (grid):")
    print_policy_fn(env, Q)

    # --- Evaluate ---
    sr = eval_fn(env, Q, episodes=eval_episodes, max_steps=max_steps_eval, seed=999)
    print(f"\nFinal greedy success rate over {eval_episodes} episodes: {sr:.3f}")

    # --- Render one episode (optional) ---
    if render_one_episode:
        renderer = FrozenLakeMatplotlibRenderer(
            env,
            bg_image_path=None,
            pause=render_pause,
            title=f"FrozenLake {n}x{n} (Greedy after Q-learning)"
        )
        run_episode_with_render(env, Q, renderer, greedy_action_fn, max_steps=max_steps_eval, pause=render_pause)


### Environment Holes setup 

In [10]:
# --- Common setup: holes ---
# Task 1: 4x4 fixed holes
holes_4x4 = {(1, 1), (1, 3), (2, 3), (3, 0)}

# Task 2: 10x10 random solvable holes
holes_10x10 = generate_random_solvable_holes(
    n=10,
    hole_ratio=0.25,
    seed=123,
    start=(0, 0),
    goal=(9, 9),
    max_tries=5000
)

print("holes_4x4 =", holes_4x4)
print("holes_10x10 count =", len(holes_10x10))
print("holes_10x10 =", holes_10x10)

holes_4x4 = {(2, 3), (1, 1), (1, 3), (3, 0)}
holes_10x10 count = 25
holes_10x10 = {(4, 0), (3, 4), (8, 6), (0, 5), (1, 9), (7, 4), (6, 2), (6, 8), (5, 6), (9, 4), (0, 1), (1, 2), (2, 7), (6, 1), (7, 3), (6, 7), (4, 1), (4, 4), (3, 8), (8, 4), (5, 8), (0, 3), (1, 4), (0, 6), (1, 7)}


## Task 1:  First-visit Monte Carlo control without exploring starts

###  Experiment 1: Deterministic Transition, Monte Carlo (task 1), 4×4 - (detMC4)

- When multiple actions had identical Q-values during the argmax operation, ties were resolved deterministically by always selecting the leftmost action; no randomness was introduced in tie-breaking.

- Epsilon Value is fixed
	
- The action-value updates were performed using the first-visit Monte Carlo method, where only the return from the first occurrence of each state–action pair in an episode was used - Many-visit updates were not applied; 

- Training episodes set upto 30,000 and its evaluation is for every 3,000



In [11]:
run_mc_experiment(
    env_class=FrozenLakeEnvDet,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    mc_control_fn=mc_control_base,
    greedy_action_fn=greedy_action_base,
    print_policy_fn=print_policy_base,
    eval_fn=eval_greedy_base,
    mc_kwargs={"epsilon": 0.10},
    train_episodes=30000,
    gamma=0.99,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={}
)


Running MC experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
MC implementation: monte_carlo.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon': 0.1}
[MC FV] episode=3000 | greedy success_rate=0.000
[MC FV] episode=6000 | greedy success_rate=0.000
[MC FV] episode=9000 | greedy success_rate=0.000
[MC FV] episode=12000 | greedy success_rate=1.000
[MC FV] episode=15000 | greedy success_rate=1.000
[MC FV] episode=18000 | greedy success_rate=1.000
[MC FV] episode=21000 | greedy success_rate=1.000
[MC FV] episode=24000 | greedy success_rate=1.000
[MC FV] episode=27000 | greedy success_rate=1.000
[MC FV] episode=30000 | greedy success_rate=1.000

Final greedy policy (grid):
S L L L
D H U H
R D L H
H R R G

Final greedy success rate over 2000 episodes: 1.000
[Renderer] Image not found: bg_images/robot.png (will fallback to markers if needed)
Initial state_id: 0 pos: (0, 0)
Agent at: (row=0, col=0) | state_id=0
---------
|S F F F|
|F H 

Explanation for Experiment 1:

Based on the success rate - (Training is happened for 30,000 episodes and success rate is evaluated after every 3000 epsisodes):
- For the first ~9k episodes, learned Q-table produces a greedy policy that never reaches the goal
- Between 9k–12k episodes, the agent finally discovers at least one successful trajectory to the goal. 
- Sharp performance jump (success rate 100%) occurs because I considered the deterministic transitions
- Learning is delayed due to sparse terminal rewards and the lack of exploring starts, which limits early state–action exploration.

Based on the final Greedy policy printed:
- 	At each state, the indicated action represents the action with the highest learned Q-value
- Some states indicate LEFT as the greedy action, which may appear unintuitive. This occurs because these states were rarely or never visited during successful episodes, resulting in their Q-values remaining at zero. When all Q-values are equal, the greedy_action() function resolves ties by selecting the first maximum action, which is LEFT (Assumption 1).

###  Experiment 2: Deterministic Transition, Monte Carlo (task 1), 10×10 - (detMC10)

- Extended version of the experiment 1 for 10×10 grid
- Training was done upto 80,000 and Epidoes were evaluated after 10,000 

In [12]:
run_mc_experiment(
    env_class=FrozenLakeEnvDet,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    mc_control_fn=mc_control_base,
    greedy_action_fn=greedy_action_base,
    print_policy_fn=print_policy_base,
    eval_fn=eval_greedy_base,
    mc_kwargs={"epsilon": 0.10},
    train_episodes=80000,
    gamma=0.99,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running MC experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
MC implementation: monte_carlo.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon': 0.1}
[MC FV] episode=10000 | greedy success_rate=0.000
[MC FV] episode=20000 | greedy success_rate=0.000
[MC FV] episode=30000 | greedy success_rate=0.000
[MC FV] episode=40000 | greedy success_rate=0.000
[MC FV] episode=50000 | greedy success_rate=0.000
[MC FV] episode=60000 | greedy success_rate=0.000
[MC FV] episode=70000 | greedy success_rate=0.000
[MC FV] episode=80000 | greedy success_rate=0.000

Final greedy policy (grid):
S H L H L H H L L L
D D H D H L L H L H
R L L L L R L H L L
U U U U H D L L H L
H H U U H L L L L L
D D R D L L H L H L
L H H L L L L H H L
L L L H H L L L L L
L L L L H L H L L L
L L L L H L L L L G

Final greedy success rate over 2000 episodes: 0.000
[Renderer] Image not found: bg_images/robot.png (will fallback to markers if needed)
Initial state_id: 0 p

Explanations of Experiment 2:

- Greedy success rate remains 0.00 throughout training, even after 80,000 episodes.
- MC fails to scale to large grids (10×10) under sparse rewards and fixed starts
- It might be due to extreme sparsity of positive reward and also MC control requires full episodes to update Q-values

From the final Greedy policy:
- There is no single continuous directed path from Start to Goal. 

During episode execution, the robot was observed to oscillate between two grid cells as a strategy to avoid falling into a hole, thereby preventing negative rewards.

###  Experiment 3: Probablistic Transition - Slippery Environment, Monte Carlo (task 1), 4×4 - (probMC4)

- Same assumptions as Experiment 1: Epsilon is fixed and Tie Breaking is biased towards the lower indexed actions
- Training episodes set upto 30,000 and its evaluation is for every 3,000

In [13]:
run_mc_experiment(
    env_class=FrozenLakeEnvProb,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    mc_control_fn=mc_control_base,
    greedy_action_fn=greedy_action_base,
    print_policy_fn=print_policy_base,
    eval_fn=eval_greedy_base,
    mc_kwargs={"epsilon": 0.10},
    train_episodes=30000,
    gamma=0.99,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={"slip_prob": 0.2}
)


Running MC experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={'slip_prob': 0.2}
MC implementation: monte_carlo.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon': 0.1}
[MC FV] episode=3000 | greedy success_rate=0.755
[MC FV] episode=6000 | greedy success_rate=0.735
[MC FV] episode=9000 | greedy success_rate=0.750
[MC FV] episode=12000 | greedy success_rate=0.685
[MC FV] episode=15000 | greedy success_rate=0.770
[MC FV] episode=18000 | greedy success_rate=0.730
[MC FV] episode=21000 | greedy success_rate=0.685
[MC FV] episode=24000 | greedy success_rate=0.715
[MC FV] episode=27000 | greedy success_rate=0.765
[MC FV] episode=30000 | greedy success_rate=0.800

Final greedy policy (grid):
S L U U
D H U H
R D D H
H R R G

Final greedy success rate over 2000 episodes: 0.760
[Renderer] Image not found: bg_images/robot.png (will fallback to markers if needed)
Initial state_id: 0 pos: (0, 0)
Agent at: (row=0, col=0) | state_id=0
---------

Explanations for Experiment 3:

- For the probabilistic Frozen Lake, the greedy success rate during Monte Carlo training does not increase monotonically but exhibits oscillations.
- This behavior is expected due to the high variance of Monte Carlo return estimates under stochastic transitions and the continued exploration induced by the ε-greedy behavior policy.
- Despite these fluctuations, the final success rate converges to approximately 0.8, which is close to the optimal achievable performance for the slippery environment.

###  Experiment 4: Probablistic Transition - Slippery Environment, Monte Carlo (task 1), 4×4 - 80k Epsiodes ----> (probMC4-moreTrain)

- This experiment extends Experiment 3 by increasing the number of training episodes to 80,000, with success evaluated every 10,000 episodes. 
- The objective is to examine whether a larger training budget improves the success rate of Monte Carlo methods in a slippery environment.

In [14]:
run_mc_experiment(
    env_class=FrozenLakeEnvProb,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    mc_control_fn=mc_control_base,
    greedy_action_fn=greedy_action_base,
    print_policy_fn=print_policy_base,
    eval_fn=eval_greedy_base,
    mc_kwargs={"epsilon": 0.10},
    train_episodes=80000,
    gamma=0.99,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={"slip_prob": 0.2}
)


Running MC experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={'slip_prob': 0.2}
MC implementation: monte_carlo.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon': 0.1}
[MC FV] episode=10000 | greedy success_rate=0.755
[MC FV] episode=20000 | greedy success_rate=0.755
[MC FV] episode=30000 | greedy success_rate=0.735
[MC FV] episode=40000 | greedy success_rate=0.730
[MC FV] episode=50000 | greedy success_rate=0.770
[MC FV] episode=60000 | greedy success_rate=0.755
[MC FV] episode=70000 | greedy success_rate=0.700
[MC FV] episode=80000 | greedy success_rate=0.795

Final greedy policy (grid):
S L D U
D H D H
R D D H
H R R G

Final greedy success rate over 2000 episodes: 0.742
[Renderer] Image not found: bg_images/robot.png (will fallback to markers if needed)
Initial state_id: 0 pos: (0, 0)
Agent at: (row=0, col=0) | state_id=0
---------
|S F F F|
|F H F H|
|F F F H|
|H F F G|
---------
[t=0] a=D (1)  ns=4  r=0.0  done=False  pos=(1

Explanation for Experiment 4:

- Increasing the number of Monte Carlo training episodes from 30,000 to 80,000 in the probabilistic environment did not result in monotonic performance improvement.
- Instead, the greedy success rate continued to oscillate around a stable value (~0.75–0.80)
- This indicates early saturation of policy quality and highlights a key limitation of Monte Carlo methods in stochastic environments.

When comparing the final Greedy policy of Experiment 1 (deterministic Transition) and Experiment 4 (Slippery Environment):
- Once the agent is close enough to the goal and away from holes, both environments agree on the optimal behavior.
- The difference is only in risky regions (near holes) : Stochastic policy optimizes survival under action noise.
- In the deterministic environment, the learned greedy policy follows a direct shortest path to the goal. In contrast, under stochastic transitions, the policy becomes noticeably more conservative, favoring actions that reduce the probability of catastrophic slips into holes.

###  Experiment 5: Probablistic Transition - Slippery Environment, Monte Carlo (task 1), 10×10 - 80k Epsiodes ----> (probMC10)

- This experiment is an extended version of experiment 4 to a 10×10 grid 

In [15]:
run_mc_experiment(
    env_class=FrozenLakeEnvProb,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    mc_control_fn=mc_control_base,
    greedy_action_fn=greedy_action_base,
    print_policy_fn=print_policy_base,
    eval_fn=eval_greedy_base,
    mc_kwargs={"epsilon": 0.10},
    train_episodes=80000,
    gamma=0.99,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={"slip_prob": 0.2}
)


Running MC experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={'slip_prob': 0.2}
MC implementation: monte_carlo.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon': 0.1}
[MC FV] episode=10000 | greedy success_rate=0.000
[MC FV] episode=20000 | greedy success_rate=0.000
[MC FV] episode=30000 | greedy success_rate=0.000
[MC FV] episode=40000 | greedy success_rate=0.000
[MC FV] episode=50000 | greedy success_rate=0.000
[MC FV] episode=60000 | greedy success_rate=0.000
[MC FV] episode=70000 | greedy success_rate=0.000
[MC FV] episode=80000 | greedy success_rate=0.000

Final greedy policy (grid):
S H L H L H H L L L
D D H D H R R H L H
L L D D L L D H L L
U U L L H R L D H L
H H U L H D L D L L
D L L U L L H D H L
L H H R L U L H H L
R D L H H R L L L L
D L L L H D H L L L
D L D L H L L L L G

Final greedy success rate over 2000 episodes: 0.000
[Renderer] Image not found: bg_images/robot.png (will fallback to markers if needed)
Initi

Explanation for Experiment 5:
- Similar to the deterministic transition environment, this experiment fails to converge to a valid start-to-goal trajectory for the 10×10 grid. 
- The observations are consistent with those from Experiment 2 (10×10 deterministic case)
- The greedy success rate remains 0.00 throughout training, even after 80,000 episodes.
- Monte Carlo control fails to scale to larger grids (10×10) under sparse rewards and fixed start states.
- Monte Carlo control fails to scale to larger grids (10×10) under sparse rewards and fixed start states.

###  Experiment 6: Deterministic Transition, Monte Carlo (task 1), 4×4 - (detMCup4)

- Random tie-breaking was used for greedy action selection to avoid systematic bias toward lower-indexed actions (Left side) during early learning when multiple actions have identical value estimates.

- An exponentially decaying ε-greedy strategy was adopted, starting with a high exploration rate and gradually reducing it to a fixed minimum:
"epsilon_start": 0.40 ----- "epsilon_min": 0.15 ------ "epsilon_decay_rate": 1.5



In [16]:
run_mc_experiment(
    env_class=FrozenLakeEnvDet,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    mc_control_fn=mc_control_up,
    greedy_action_fn=greedy_action_up,
    print_policy_fn=print_policy_up,
    eval_fn=eval_greedy_up,
    mc_kwargs={
        "epsilon_start": 0.40,
        "epsilon_min": 0.15,
        "epsilon_decay_type": "exp",   # "exp" or "linear"
        "epsilon_decay_rate": 1.5,     # higher = faster decay
        "epsilon_decay_fraction": 0.8  # only used if decay_type == "linear"
    },
    train_episodes=30000,
    gamma=0.99,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={}
)


Running MC experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
MC implementation: monte_carlo_update.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon_start': 0.4, 'epsilon_min': 0.15, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 1.5, 'epsilon_decay_fraction': 0.8}
[MC FV] episode=3000 | eps=0.3652 | greedy success_rate=1.000
[MC FV] episode=6000 | eps=0.3352 | greedy success_rate=1.000
[MC FV] episode=9000 | eps=0.3094 | greedy success_rate=1.000
[MC FV] episode=12000 | eps=0.2872 | greedy success_rate=1.000
[MC FV] episode=15000 | eps=0.2681 | greedy success_rate=1.000
[MC FV] episode=18000 | eps=0.2516 | greedy success_rate=1.000
[MC FV] episode=21000 | eps=0.2375 | greedy success_rate=1.000
[MC FV] episode=24000 | eps=0.2253 | greedy success_rate=1.000
[MC FV] episode=27000 | eps=0.2148 | greedy success_rate=1.000
[MC FV] episode=30000 | eps=0.2058 | greedy success_rate=1.000

Final greedy policy (grid):
S L L L
D H D 

Explanation for Experiment 6:

- ε is decaying smoothly
- Greedy success rate is exactly 1.000 from very early on (its possible, because of the deterministic environment)
- While the optimal start-to-goal trajectory is identical to that of Experiment 1, the resulting greedy policy is more realistic in sparsely visited regions, reflecting improved action selection beyond the main optimal path, due to the early exploration.

###  Experiment 7: Deterministic Transition, Monte Carlo (task 1), 10×10 - (detMCup10)

- This experiment is an extended version of Experiment 6 on a 10×10 grid with deterministic transitions, random tie-breaking, and a decaying ε-greedy strategy. 

In [17]:
run_mc_experiment(
    env_class=FrozenLakeEnvDet,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    mc_control_fn=mc_control_up,
    greedy_action_fn=greedy_action_up,
    print_policy_fn=print_policy_up,
    eval_fn=eval_greedy_up,
    mc_kwargs={
        "epsilon_start": 0.6,
        "epsilon_min": 0.2,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 1.0,
        "epsilon_decay_fraction": 0.8
    },
    train_episodes=80000,
    gamma=0.99,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running MC experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
MC implementation: monte_carlo_update.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon_start': 0.6, 'epsilon_min': 0.2, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 1.0, 'epsilon_decay_fraction': 0.8}
[MC FV] episode=10000 | eps=0.5530 | greedy success_rate=0.000
[MC FV] episode=20000 | eps=0.5115 | greedy success_rate=0.000
[MC FV] episode=30000 | eps=0.4749 | greedy success_rate=0.000
[MC FV] episode=40000 | eps=0.4426 | greedy success_rate=0.000
[MC FV] episode=50000 | eps=0.4141 | greedy success_rate=0.000
[MC FV] episode=60000 | eps=0.3889 | greedy success_rate=0.000
[MC FV] episode=70000 | eps=0.3667 | greedy success_rate=0.000
[MC FV] episode=80000 | eps=0.3472 | greedy success_rate=0.000

Final greedy policy (grid):
S H U H U H H L R U
D L H D H D D H D H
L L L L L L L H L R
U U U L H R U U H U
H H U L H D U L U U
R R U U L L H D H D
L H H U U L L H

Explanations for Experiment 7:

- Despite these algorithmic improvements, the method failed to converge to a 10×10 grid, as the greedy policy success rate remained zero even after 80,000 training episodes.

###  Experiment 8: Probablistic Transition - Slippery Environment, Monte Carlo (task 1), 4×4 - (probMCup4)

- Same assumptions as experiment 6: random tie-breaking, and a decaying ε-greedy strategy. 

## probMCup4 (Slippery, MC updated, 4×4)

In [18]:
run_mc_experiment(
    env_class=FrozenLakeEnvProb,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    mc_control_fn=mc_control_up,
    greedy_action_fn=greedy_action_up,
    print_policy_fn=print_policy_up,
    eval_fn=eval_greedy_up,
    mc_kwargs={
        "epsilon_start": 0.40,
        "epsilon_min": 0.15,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 1.5,
        "epsilon_decay_fraction": 0.8
    },
    train_episodes=30000,
    gamma=0.99,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={"slip_prob": 0.2}
)


Running MC experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={'slip_prob': 0.2}
MC implementation: monte_carlo_update.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon_start': 0.4, 'epsilon_min': 0.15, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 1.5, 'epsilon_decay_fraction': 0.8}
[MC FV] episode=3000 | eps=0.3652 | greedy success_rate=0.780
[MC FV] episode=6000 | eps=0.3352 | greedy success_rate=0.755
[MC FV] episode=9000 | eps=0.3094 | greedy success_rate=0.735
[MC FV] episode=12000 | eps=0.2872 | greedy success_rate=0.705
[MC FV] episode=15000 | eps=0.2681 | greedy success_rate=0.690
[MC FV] episode=18000 | eps=0.2516 | greedy success_rate=0.730
[MC FV] episode=21000 | eps=0.2375 | greedy success_rate=0.730
[MC FV] episode=24000 | eps=0.2253 | greedy success_rate=0.710
[MC FV] episode=27000 | eps=0.2148 | greedy success_rate=0.715
[MC FV] episode=30000 | eps=0.2058 | greedy success_rate=0.770

Final greedy policy (grid)

Explanation for Experiment 8: 

- The obtained results are much more similar to Experiment 3 and 4 - Probablistic Environment, but with fixed epsilon
- During episode execution, some slippery movements were observed, which do not correspond to the actions prescribed by the final greedy policy.

###  Experiment 9: Probablistic Transition - Slippery Environment, Monte Carlo (task 1), 10×10 - (probMCup10)

In [19]:
run_mc_experiment(
    env_class=FrozenLakeEnvProb,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    mc_control_fn=mc_control_up,
    greedy_action_fn=greedy_action_up,
    print_policy_fn=print_policy_up,
    eval_fn=eval_greedy_up,
    mc_kwargs={
        "epsilon_start": 0.6,
        "epsilon_min": 0.2,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 1.0,
        "epsilon_decay_fraction": 0.8
    },
    train_episodes=80000,
    gamma=0.99,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={"slip_prob": 0.2}
)


Running MC experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={'slip_prob': 0.2}
MC implementation: monte_carlo_update.mc_control_first_visit_no_exploring_starts | mc_kwargs={'epsilon_start': 0.6, 'epsilon_min': 0.2, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 1.0, 'epsilon_decay_fraction': 0.8}
[MC FV] episode=10000 | eps=0.5530 | greedy success_rate=0.000
[MC FV] episode=20000 | eps=0.5115 | greedy success_rate=0.000
[MC FV] episode=30000 | eps=0.4749 | greedy success_rate=0.000
[MC FV] episode=40000 | eps=0.4426 | greedy success_rate=0.000
[MC FV] episode=50000 | eps=0.4141 | greedy success_rate=0.000
[MC FV] episode=60000 | eps=0.3889 | greedy success_rate=0.000
[MC FV] episode=70000 | eps=0.3667 | greedy success_rate=0.000
[MC FV] episode=80000 | eps=0.3472 | greedy success_rate=0.000

Final greedy policy (grid):
S H U H U H H L R U
D L H D H R L H D H
L L L L L U L H L R
U U L L H R D R H U
H H U L H U R U U U
D L U U R D H L H D

Explanation for Experiment 9:

- Greedy success rate remains 0.00 throughout training, even after 80,000 episodes similar to previous 10×10 grid experiments

## Task 2: SARSA with an ϵ-greedy behavior policy

- SARSA: Unlike Monte Carlo methods, SARSA does not require waiting for an episode to terminate to update value estimates. Instead, the action-value function is updated at every time step using the immediate reward and the subsequent state–action pair (SARSA performs on-policy updates at every time step using the immediate reward and the next state–action pair selected by the behavior policy).

###  Experiment 10: Deterministic Transition, SARSA (task 2), 4×4 - (detSARSA4)

- Random tie-breaking was used for greedy action selection to avoid systematic bias toward lower-indexed actions (Left side) during early learning when multiple actions have identical value estimates.
- Fixed epsilon for exploration (0.1)


In [20]:
run_sarsa_experiment(
    env_class=FrozenLakeEnvDet,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    sarsa_control_fn=sarsa_control_up,
    greedy_action_fn=greedy_action_sarsa,
    print_policy_fn=print_policy_sarsa,
    eval_fn=eval_greedy_sarsa,
    sarsa_kwargs={
        "epsilon": 0.10,
        "use_epsilon_decay": False,
    },
    train_episodes=30000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={}
)


Running SARSA experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
SARSA implementation: sarsa.sarsa_control_epsilon_greedy | sarsa_kwargs={'epsilon': 0.1, 'use_epsilon_decay': False}
[SARSA] episode=3000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=6000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=9000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=12000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=15000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=18000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=21000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=24000 | eps=0.1000 | greedy success_rate=0.000
[SARSA] episode=27000 | eps=0.1000 | greedy success_rate=1.000
[SARSA] episode=30000 | eps=0.1000 | greedy success_rate=1.000

Final greedy policy (grid):
S L R L
D H D H
R R D H
H R R G

Final greedy success rate over 2000 episodes: 1.000
[Renderer] Image not found: bg_im

Explanation for Experiment 10:

- The success rate converges to 1 within fewer than 3,000 training episodes, which is possible due to the deterministic nature of the state transitions.
- But it seems like it has converged quicker than Monte-Carlo technique
- Final Greedy policy is almost similar to Monte Carlo results

Reason for the same observation:
- Environment is deterministic (no slip) and it’s a small state space (4×4) – Therefore, SARSA doesn’t gain any special robustness advantage here 
- Differences only show up in how fast, not where they end



###  Experiment 11: Deterministic Transition, SARSA (task 2), 10×10 - (detSARSA10)

- An exponentially decaying ε-greedy strategy was adopted

In [24]:
run_sarsa_experiment(
    env_class=FrozenLakeEnvDet,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    sarsa_control_fn=sarsa_control_up,
    greedy_action_fn=greedy_action_sarsa,
    print_policy_fn=print_policy_sarsa,
    eval_fn=eval_greedy_sarsa,
    sarsa_kwargs={
        "use_epsilon_decay": True,
        "epsilon_start": 0.6,
        "epsilon_min": 0.2,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 1.0,
        "epsilon_decay_fraction": 0.8
    },
    train_episodes=80000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running SARSA experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
SARSA implementation: sarsa.sarsa_control_epsilon_greedy | sarsa_kwargs={'use_epsilon_decay': True, 'epsilon_start': 0.6, 'epsilon_min': 0.2, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 1.0, 'epsilon_decay_fraction': 0.8}
[SARSA] episode=10000 | eps=0.5530 | greedy success_rate=0.000
[SARSA] episode=20000 | eps=0.5115 | greedy success_rate=0.000
[SARSA] episode=30000 | eps=0.4749 | greedy success_rate=0.000
[SARSA] episode=40000 | eps=0.4426 | greedy success_rate=0.000
[SARSA] episode=50000 | eps=0.4141 | greedy success_rate=0.000
[SARSA] episode=60000 | eps=0.3889 | greedy success_rate=0.000
[SARSA] episode=70000 | eps=0.3667 | greedy success_rate=0.000
[SARSA] episode=80000 | eps=0.3472 | greedy success_rate=1.000

Final greedy policy (grid):
S H U H U H H R U R
D D H D H D D H U H
R R D L R D D H U L
U R D L H R R D H D
H H D D H D R R R D
D R R D D D H U H D
D H H 

Explanation for Experiment 11:

- The exploration rate ε decays from approximately 0.6 to 0.35 over 80,000 episodes.
- The greedy success rate remains at 0.000 until around 70,000 episodes and increases to 1.000 only near the end of training.
- As a result, SARSA initially experiences mostly unsuccessful episodes; with sparse terminal rewards, Q-values close to the goal are seldom reinforced, and learning is dominated by negative updates in upstream states.


Monte Carlo vs. SARSA Convergence
- Monte Carlo does not converge within 80,000 training episodes, whereas SARSA converges under the same setting.

Value Propagation Mechanism
- Monte Carlo: Value updates occur only after an episode terminates.
- SARSA: Value updates are performed at every time step based on the action it selected.

As a result, SARSA can:
- Learn that certain actions are less detrimental than others, even if they do not immediately lead to the goal.
- Gradually build a smooth value gradient toward the goal state.
- Make learning progress even when successful goal-reaching episodes are rare.

In contrast, Monte Carlo methods cannot propagate such intermediate value information because updates depend solely on complete episodes reaching terminal states.

In the 10×10 grid with sparse rewards and a high hole density, Monte Carlo control fails to converge within 80,000 episodes because it relies exclusively on complete successful trajectories to update state–action values. Such trajectories are extremely rare under ε-greedy exploration, resulting in insufficient positive return signals. In contrast, SARSA employs temporal-difference bootstrapping, allowing value information from near-goal states to propagate incrementally even in the absence of frequent successful episodes. As ε decays, SARSA stabilizes its policy and eventually converges to a successful greedy policy, explaining the observed sharp transition from zero to full success near the end of training.


The final learned policy (Final Greedy Policy Grid) highlights a path from the start to the goal

It is not the shortest path. Instead, the trajectory exhibits zig-zagging behavior near holes.

This occurs because SARSA learns the value of the actions it actually executes under the behavior policy, rather than the actions that would be optimal under an idealized greedy policy.

- SARSA implicitly penalizes risky states in the vicinity of holes.
- The observed zig-zag patterns are not noise, but a form of risk smoothing.
- SARSA converges to a risk-averse policy.


###  Experiment 12: Deterministic Transition, SARSA (task 2), 10×10 - (detSARSA10)

- Similar to Experiment 11, but with the less starting epsilon value compared to Experiment 11

In [26]:
run_sarsa_experiment(
    env_class=FrozenLakeEnvDet,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    sarsa_control_fn=sarsa_control_up,
    greedy_action_fn=greedy_action_sarsa,
    print_policy_fn=print_policy_sarsa,
    eval_fn=eval_greedy_sarsa,
    sarsa_kwargs={
        "use_epsilon_decay": True,
        "epsilon_start": 0.4,
        "epsilon_min": 0.3,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 0.6,
        "epsilon_decay_fraction": 0.6
    },
    train_episodes=80000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running SARSA experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
SARSA implementation: sarsa.sarsa_control_epsilon_greedy | sarsa_kwargs={'use_epsilon_decay': True, 'epsilon_start': 0.4, 'epsilon_min': 0.3, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 0.6, 'epsilon_decay_fraction': 0.6}
[SARSA] episode=10000 | eps=0.3928 | greedy success_rate=1.000
[SARSA] episode=20000 | eps=0.3861 | greedy success_rate=0.000
[SARSA] episode=30000 | eps=0.3799 | greedy success_rate=0.000
[SARSA] episode=40000 | eps=0.3741 | greedy success_rate=0.000
[SARSA] episode=50000 | eps=0.3687 | greedy success_rate=1.000
[SARSA] episode=60000 | eps=0.3638 | greedy success_rate=0.000
[SARSA] episode=70000 | eps=0.3592 | greedy success_rate=1.000
[SARSA] episode=80000 | eps=0.3549 | greedy success_rate=1.000

Final greedy policy (grid):
S H U H U H H L R U
D D H D H D D H U H
R D D R R R D H R D
U R D L H R D D H D
H H D D H D R R R D
D R R D R D H U H R
D H H 

- Compared to a larger epsilon value where the robot explores more heavily and exhibits frequent oscillations, the algorithm converges earlier when a smaller epsilon is used.
- SARSA sometimes produces a fully successful greedy policy - Then later loses it - Then recovers and stabilizes
- SARSA is not trying to optimize a greedy policy - It is optimizing an ε-soft policy - Greedy success rate during training is not a monotonic metric for SARSA
- The final greedy policy is similar to that of Experiment 11 and exhibits a zig-zag trajectory, reflecting the risk-averse nature of SARSA’s on-policy learning.


###  Experiment 13: Probablistic Transition - Slippery Environment, SARSA (task 2), 4×4

Experiment setup is similar to Experiment 10, but with the probabilistic Transitions (Slippery environment) – Fixed epsilon

In [27]:
run_sarsa_experiment(
    env_class=FrozenLakeEnvProb,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    sarsa_control_fn=sarsa_control_up,
    greedy_action_fn=greedy_action_sarsa,
    print_policy_fn=print_policy_sarsa,
    eval_fn=eval_greedy_sarsa,
    sarsa_kwargs={
        "epsilon": 0.10,
        "use_epsilon_decay": False,
    },
    train_episodes=30000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={}
)


Running SARSA experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
SARSA implementation: sarsa.sarsa_control_epsilon_greedy | sarsa_kwargs={'epsilon': 0.1, 'use_epsilon_decay': False}
[SARSA] episode=3000 | eps=0.1000 | greedy success_rate=0.745
[SARSA] episode=6000 | eps=0.1000 | greedy success_rate=0.840
[SARSA] episode=9000 | eps=0.1000 | greedy success_rate=0.820
[SARSA] episode=12000 | eps=0.1000 | greedy success_rate=0.870
[SARSA] episode=15000 | eps=0.1000 | greedy success_rate=0.745
[SARSA] episode=18000 | eps=0.1000 | greedy success_rate=0.835
[SARSA] episode=21000 | eps=0.1000 | greedy success_rate=0.705
[SARSA] episode=24000 | eps=0.1000 | greedy success_rate=0.865
[SARSA] episode=27000 | eps=0.1000 | greedy success_rate=0.810
[SARSA] episode=30000 | eps=0.1000 | greedy success_rate=0.785

Final greedy policy (grid):
S U D L
D H D H
R D L H
H R R G

Final greedy success rate over 2000 episodes: 0.760
[Renderer] Image not found: bg_im

Explanation for Experiment 13:

Observations are almost similar to Monte carlo results, but there success converge between 0.75 – 0.8, but here success converge between 0.75 – 0.85 which is bit higher than Monte-Carlo, but due to slippery nature, we couldn’t able to expect the convergence rate of 1. 

###  Experiment 14: Probablistic Transition - Slippery Environment, SARSA (task 2), 10×10

- Extended experiment of 13, for 10×10 grid - Probablistic action - Decaying epsilon 

In [33]:
run_sarsa_experiment(
    env_class=FrozenLakeEnvProb,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    sarsa_control_fn=sarsa_control_up,
    greedy_action_fn=greedy_action_sarsa,
    print_policy_fn=print_policy_sarsa,
    eval_fn=eval_greedy_sarsa,
    sarsa_kwargs={
        "use_epsilon_decay": True,
        "epsilon_start": 0.3,
        "epsilon_min": 0.1,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 0.6,
        "epsilon_decay_fraction": 0.6
    },
    train_episodes=80000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running SARSA experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
SARSA implementation: sarsa.sarsa_control_epsilon_greedy | sarsa_kwargs={'use_epsilon_decay': True, 'epsilon_start': 0.3, 'epsilon_min': 0.1, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 0.6, 'epsilon_decay_fraction': 0.6}
[SARSA] episode=10000 | eps=0.2855 | greedy success_rate=0.010
[SARSA] episode=20000 | eps=0.2721 | greedy success_rate=0.465
[SARSA] episode=30000 | eps=0.2597 | greedy success_rate=0.305
[SARSA] episode=40000 | eps=0.2482 | greedy success_rate=0.000
[SARSA] episode=50000 | eps=0.2375 | greedy success_rate=0.000
[SARSA] episode=60000 | eps=0.2275 | greedy success_rate=0.525
[SARSA] episode=70000 | eps=0.2183 | greedy success_rate=0.130
[SARSA] episode=80000 | eps=0.2098 | greedy success_rate=0.355

Final greedy policy (grid):
S H U H U H H L R U
D D H D H D D H D H
R R D D R D L H U D
U U R D H R D D H D
H H D L H D R R R D
D L R R R D H U H D
D H H 

###  Experiment 15: Probablistic Transition - Slippery Environment, SARSA (task 2), 10×10

- Extended experiment of 14, for 10×10 grid - Probablistic action - Decaying epsilon - But with the more Training episodes - 160K 

In [39]:
run_sarsa_experiment(
    env_class=FrozenLakeEnvProb,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    sarsa_control_fn=sarsa_control_up,
    greedy_action_fn=greedy_action_sarsa,
    print_policy_fn=print_policy_sarsa,
    eval_fn=eval_greedy_sarsa,
    sarsa_kwargs={
        "use_epsilon_decay": True,
        "epsilon_start": 0.25,
        "epsilon_min": 0.1,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 0.6,
        "epsilon_decay_fraction": 0.6
    },
    train_episodes=160000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=20000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running SARSA experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
SARSA implementation: sarsa.sarsa_control_epsilon_greedy | sarsa_kwargs={'use_epsilon_decay': True, 'epsilon_start': 0.25, 'epsilon_min': 0.1, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 0.6, 'epsilon_decay_fraction': 0.6}
[SARSA] episode=20000 | eps=0.2392 | greedy success_rate=0.470
[SARSA] episode=40000 | eps=0.2291 | greedy success_rate=0.275
[SARSA] episode=60000 | eps=0.2198 | greedy success_rate=0.475
[SARSA] episode=80000 | eps=0.2111 | greedy success_rate=0.540
[SARSA] episode=100000 | eps=0.2031 | greedy success_rate=0.585
[SARSA] episode=120000 | eps=0.1956 | greedy success_rate=0.450
[SARSA] episode=140000 | eps=0.1887 | greedy success_rate=0.535
[SARSA] episode=160000 | eps=0.1823 | greedy success_rate=0.595

Final greedy policy (grid):
S H U H U H H L R U
D D H D H D D H D H
U R D D L D L H R D
U U D L H R R D H R
H H R D H R U R R D
D L R R D D H U H D
D

Explanation for Experiment 15:

Cannot able to converge, even though Training has been increased to 160K.

## Task 3:  𝑄-learning with an ϵ-greedy behavior policy.

###  Experiment 16: Deterministic Transition, Q-learning (task 3), 4×4 - (detQL4)

- Random tie-breaking was used for greedy action selection to avoid systematic bias toward lower-indexed actions (Left side) during early learning when multiple actions have identical value estimates.
- Fixed epsilon for exploration (0.1)


In [45]:
run_q_learning_experiment(
    env_class=FrozenLakeEnvDet,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    q_learning_control_fn=q_learning_control_up,
    greedy_action_fn=greedy_action_q,
    print_policy_fn=print_policy_q,
    eval_fn=eval_greedy_q,
    q_learning_kwargs={
        "epsilon": 0.10,
        "use_epsilon_decay": False,
    },
    train_episodes=30000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=3000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={}
)


Running Q-learning experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
Q-learning implementation: q_learning.q_learning_control_epsilon_greedy | q_learning_kwargs={'epsilon': 0.1, 'use_epsilon_decay': False}
[Q-Learning] episode=3000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=6000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=9000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=12000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=15000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=18000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=21000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=24000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=27000 | eps=0.1000 | greedy success_rate=1.000
[Q-Learning] episode=30000 | eps=0.1000 | greedy success_rate=1.000

Final greedy policy (grid):
S L L L
D H D H
R D D H
H R R G

Final gree

Explanation for Experiment 16:

- Since its deterministic actions and it has small state soace 4*4, this output looks much similar to Monte Carlo and SARSA
- The success rate converges to 1 within fewer than 3,000 training episodes, which is possible due to the deterministic nature of the state transitions.

###  Experiment 17: Deterministic Transition, Q-learning (task 3), 10×10 - (detQL10)

In [49]:
run_q_learning_experiment(
    env_class=FrozenLakeEnvDet,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    q_learning_control_fn=q_learning_control_up,
    greedy_action_fn=greedy_action_q,
    print_policy_fn=print_policy_q,
    eval_fn=eval_greedy_q,
    q_learning_kwargs={
        "use_epsilon_decay": True,
        "epsilon_start": 0.25,
        "epsilon_min": 0.10,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 0.6,
        "epsilon_decay_fraction": 0.6
    },
    train_episodes=80000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=10000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running Q-learning experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
Q-learning implementation: q_learning.q_learning_control_epsilon_greedy | q_learning_kwargs={'use_epsilon_decay': True, 'epsilon_start': 0.25, 'epsilon_min': 0.1, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 0.6, 'epsilon_decay_fraction': 0.6}
[Q-Learning] episode=10000 | eps=0.2392 | greedy success_rate=1.000
[Q-Learning] episode=20000 | eps=0.2291 | greedy success_rate=1.000
[Q-Learning] episode=30000 | eps=0.2198 | greedy success_rate=1.000
[Q-Learning] episode=40000 | eps=0.2111 | greedy success_rate=1.000
[Q-Learning] episode=50000 | eps=0.2031 | greedy success_rate=1.000
[Q-Learning] episode=60000 | eps=0.1956 | greedy success_rate=1.000
[Q-Learning] episode=70000 | eps=0.1887 | greedy success_rate=1.000
[Q-Learning] episode=80000 | eps=0.1823 | greedy success_rate=1.000

Final greedy policy (grid):
S H L H L H H L L L
D D H D H D L H L H
D D D D L L L H D L


Explanation for Experiment 17:

- In Deterministic environment, there is no execution risk. Once Q-learning discovers any valid shortest safe path, following it greedily always succeeds.
- Q-learning is optimistic and off-policy; this is the core reason it outperforms Monte Carlo and SARSA in this setting. During learning, Q-learning assumes optimal behavior from the next state onward and is therefore insensitive to the fact that the behavior policy is ε-greedy.
- However in SARSA, even after convergence, the learned Q-values reflect the residual ε-induced risk. Consequently, in long-horizon environments, this leads to persistent oscillations in the greedy success rate.
- Overall outcome: In the deterministic 10×10 FrozenLake environment, Q-learning rapidly converges to an optimal policy. Because transitions are deterministic and evaluation uses a purely greedy policy, the agent follows the same safe trajectory in every episode, resulting in a consistent greedy success rate of 1.0.


###  Experiment 18: Probablistic Transition - Slippery Environment, Q-learning (task 3), 4×4 - (probQL4)

In [50]:
run_q_learning_experiment(
    env_class=FrozenLakeEnvProb,
    n=4,
    holes=holes_4x4,
    env_seed=40,
    q_learning_control_fn=q_learning_control_up,
    greedy_action_fn=greedy_action_q,
    print_policy_fn=print_policy_q,
    eval_fn=eval_greedy_q,
    q_learning_kwargs={
        "epsilon": 0.10,
        "use_epsilon_decay": False,
    },
    train_episodes=60000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=100,
    eval_episodes=2000,
    max_steps_eval=100,
    verbose_every=6000,
    render_one_episode=True,
    render_pause=0.30,
    env_kwargs={}
)


Running Q-learning experiment: grid=4x4 | holes=4 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
Q-learning implementation: q_learning.q_learning_control_epsilon_greedy | q_learning_kwargs={'epsilon': 0.1, 'use_epsilon_decay': False}
[Q-Learning] episode=6000 | eps=0.1000 | greedy success_rate=0.850
[Q-Learning] episode=12000 | eps=0.1000 | greedy success_rate=0.835
[Q-Learning] episode=18000 | eps=0.1000 | greedy success_rate=0.490
[Q-Learning] episode=24000 | eps=0.1000 | greedy success_rate=0.510
[Q-Learning] episode=30000 | eps=0.1000 | greedy success_rate=0.590
[Q-Learning] episode=36000 | eps=0.1000 | greedy success_rate=0.855
[Q-Learning] episode=42000 | eps=0.1000 | greedy success_rate=0.470
[Q-Learning] episode=48000 | eps=0.1000 | greedy success_rate=0.495
[Q-Learning] episode=54000 | eps=0.1000 | greedy success_rate=0.870
[Q-Learning] episode=60000 | eps=0.1000 | greedy success_rate=0.885

Final greedy policy (grid):
S U L U
L H D H
R D L H
H R R G

Final gr

Explanation for Experiment 18:

- This output does not look similar to Monte Carlo and SARSA
- There is no single path from Start to Goal 
- loops (L <-> Boundry patterns),
- Q-learning optimizes expected return assuming future greedy actions, but training data is generated under a stochastic behavior policy.


###  Experiment 19: Probablistic Transition - Slippery Environment, Q-learning (task 3), 10×10 - (probQL10)

In [48]:
run_q_learning_experiment(
    env_class=FrozenLakeEnvProb,
    n=10,
    holes=holes_10x10,
    env_seed=123,
    q_learning_control_fn=q_learning_control_up,
    greedy_action_fn=greedy_action_q,
    print_policy_fn=print_policy_q,
    eval_fn=eval_greedy_q,
    q_learning_kwargs={
        "use_epsilon_decay": True,
        "epsilon_start": 0.25,
        "epsilon_min": 0.10,
        "epsilon_decay_type": "exp",
        "epsilon_decay_rate": 0.6,
        "epsilon_decay_fraction": 0.6
    },
    train_episodes=160000,
    gamma=0.99,
    alpha=0.10,
    max_steps_train=300,
    eval_episodes=2000,
    max_steps_eval=300,
    verbose_every=20000,
    render_one_episode=True,
    render_pause=0.10,
    env_kwargs={}
)


Running Q-learning experiment: grid=10x10 | holes=25 | hole_ratio=25.00%
Environment: FrozenLakeEnv | extra_args={}
Q-learning implementation: q_learning.q_learning_control_epsilon_greedy | q_learning_kwargs={'use_epsilon_decay': True, 'epsilon_start': 0.25, 'epsilon_min': 0.1, 'epsilon_decay_type': 'exp', 'epsilon_decay_rate': 0.6, 'epsilon_decay_fraction': 0.6}
[Q-Learning] episode=20000 | eps=0.2392 | greedy success_rate=0.000
[Q-Learning] episode=40000 | eps=0.2291 | greedy success_rate=0.000
[Q-Learning] episode=60000 | eps=0.2198 | greedy success_rate=0.000
[Q-Learning] episode=80000 | eps=0.2111 | greedy success_rate=0.000
[Q-Learning] episode=100000 | eps=0.2031 | greedy success_rate=0.000
[Q-Learning] episode=120000 | eps=0.1956 | greedy success_rate=0.000
[Q-Learning] episode=140000 | eps=0.1887 | greedy success_rate=0.000
[Q-Learning] episode=160000 | eps=0.1823 | greedy success_rate=0.000

Final greedy policy (grid):
S H L H L H H L L L
L D H D H D L H L H
L D D D L D L H 

Explanation for Experiment 19:

- It could not able to converge at least for a small value in 16,000 Training episodes
- Because the environment is large and stochastic, Q-learning’s assumption of optimal future actions makes convergence as the hardest task
